# Ejemplo 1 

### Configuración del entorno de Docker

- Crear la red para que los contenedores de Hadoop, Hive etc puedan comunicarse


In [ ]:
docker network create bigdata_network

- Desplegar un clúster de Hadoop en Docker. Utilizaremos imágenes de Docker disponibles para Hadoop y Hive en un clúster básico.
- Crea un archivo `docker-compose.yml` con las configuraciones necesarias para un clúster de Hadoop.


In [ ]:
# Yaml
version: '3'
services:
  namenode:
    image: bde2020/hadoop-namenode:2.0.0-hadoop2.7.4-java8
    container_name: namenode
    environment:
      - CLUSTER_NAME=test
    ports:
      - 9870:9870
      - 9000:9000
    networks:
      - bigdata_network

  datanode:
    image: bde2020/hadoop-datanode:2.0.0-hadoop2.7.4-java8
    container_name: datanode
    environment:
      - CLUSTER_NAME=test
      - CORE_CONF_fs_defaultFS=hdfs://namenode:9000
    networks:
      - bigdata_network

  hive:
    image: bde2020/hive:2.3.2-postgresql-metastore
    container_name: hive-server
    environment:
      - HIVE_CORE_CONF_javax_jdo_option_ConnectionURL=jdbc:postgresql://metastore:5432/metastore
    ports:
      - 10000:10000
    networks:
      - bigdata_network

  metastore:
    image: bde2020/hive-metastore-postgresql
    container_name: metastore
    environment:
      - POSTGRES_DB=metastore
      - POSTGRES_USER=hive
      - POSTGRES_PASSWORD=hive
    networks:
      - bigdata_network

networks:
  bigdata_network:
    external: true


- Iniciar el clúster de Hadoop y Hive. Ejecuta el siguiente comando en el directorio donde creaste `docker-compose.yml`. 

In [ ]:
docker-compose up -d

Este comando desplegará los contenedores necesarios para el clúster de Hadoop y el servicio de Hive.

### Ingesta y Preprocesamiento de Datos

- Preparar los datos para cargar en Hadoop. 

Supongamos que tienes un archivo `sales_data.csv` con datos de ventas que deseas cargar en HDFS.  
 
Mueve el archivo `sales_data.csv` al contenedor namenode para cargarlo en HDFS:

In [ ]:
docker cp sales_data.csv namenode:/

- Cargar el archivo en HDFS

Accede al contenedor `namenode` y carga el archivo en HDFS:

In [ ]:
docker exec -it namenode bash
hdfs dfs -mkdir -p /data/sales
hdfs dfs -put /sales_data.csv /data/sales

Esto almacena el archivo `sales_data.csv` en el sistema de archivos distribuido HDFS.

### Consultas SQL con Hive

Crear una tabla en Hive. Accede al contenedor `hive-server` para crear una tabla y cargar los datos en Hive.

In [ ]:
docker exec -it hive-server bash
hive

En el intérprete de Hive, crea una tabla para los datos de ventas:

In [ ]:
CREATE EXTERNAL TABLE IF NOT EXISTS sales_data (
    order_id STRING,
    product STRING,
    quantity INT,
    price FLOAT,
    order_date STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION '/data/sales';

Este comando configura una tabla que utiliza los datos en HDFS y permite realizar consultas sobre ella.

Realizar una consulta en Hive para agregación. Ejecuta una consulta en Hive para calcular las ventas totales por producto:

In [ ]:
SELECT product, SUM(quantity * price) AS total_sales
FROM sales_data
GROUP BY product;

Guarda el resultado como una nueva tabla o exporta los datos a HDFS para análisis posterior en Power BI.

### Visualización de Datos en Power BI

- Conectar Power BI a Hive

Abre Power BI y selecciona Obtener datos > Más.  

Escoge ODBC o JDBC como conector (asegúrate de tener el conector de Hive configurado en tu sistema).  

Usa la conexión a tu servicio de Hive en Docker (hive-server:10000) para acceder a los datos.

- Crear Visualizaciones

Crea visualizaciones, como gráficos de barras, para mostrar las ventas totales por producto.  
Usa la opción de actualización automática de Power BI para reflejar cualquier cambio en los datos de Hive.  


### Automatización y Escalabilidad

- Automatizar el flujo de datos

Puedes configurar Apache Airflow en Docker para automatizar la ingesta, transformación y carga de datos. Usa airflow para programar tareas periódicas (por ejemplo, actualizar datos en Hive y cargar resultados en Power BI).

- Escalar el entorno con Kubernetes

Si necesitas mayor capacidad de procesamiento, considera desplegar el clúster en Kubernetes.  
Usa un archivo yaml de Kubernetes para definir los despliegues y replica tus servicios de Hadoop y Hive en múltiples nodos, facilitando la distribución de carga.